In [2]:
REPO_NAME="big-data-lab"
USER_NAME="kornisch" # TODO: replace with your GitHub username

In [3]:
%env REPO_NAME=$REPO_NAME
%env USER_NAME=$USER_NAME

env: REPO_NAME=big-data-lab
env: USER_NAME=kornisch


In [4]:
%%shell
rm -rf $REPO_NAME
git clone https://github.com/$USER_NAME/$REPO_NAME.git
cd $REPO_NAME

Cloning into 'big-data-lab'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 128 (delta 38), reused 14 (delta 14), pack-reused 72 (from 3)
Receiving objects: 100% (128/128), 461.93 KiB | 23.10 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [5]:
%run $REPO_NAME/notebooks/lib/00_terraform_setup.ipynb

env: TERRAFORM_VERSION=1.9.8
env: PATH=/content/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
--2025-01-18 12:22:55--  https://releases.hashicorp.com/terraform/1.9.8/terraform_1.9.8_linux_amd64.zip
Resolving releases.hashicorp.com (releases.hashicorp.com)... 3.171.85.128, 3.171.85.65, 3.171.85.107, ...
Connecting to releases.hashicorp.com (releases.hashicorp.com)|3.171.85.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27041434 (26M) [application/zip]
Saving to: ‘terraform_1.9.8_linux_amd64.zip’

terraform_1.9.8_lin 100%[===================>]  25.79M  --.-KB/s    in 0.1s    

2025-01-18 12:22:55 (228 MB/s) - ‘terraform_1.9.8_linux_amd64.zip’ saved [27041434/27041434]

Archive:  terraform_1.9.8_linux_amd64.zip
  inflating: bin/LICENSE.txt         
  inflating: bin/terraform           
Terraform v1.9.8
on linux_amd64

Your version of Terraform is

In [6]:
%run $REPO_NAME/notebooks/conf/00_env_variables.ipynb

env: TF_VAR_ds_semester=2024l
env: TF_VAR_user_id=7042
env: TF_VAR_billing_account=01754E-828141-E8C644
env: TF_VAR_region=europe-west1
env: TF_VAR_project_name=ds-2024l-7042
env: GIT_BRANCH=colab-dev


In [7]:
# Authenticate to the Google account
from google.colab import auth
auth.authenticate_user()

### Set the remote bucket for storing Terraform state


In [8]:
import os
TF_VAR_user_id = os.getenv("TF_VAR_user_id")
TF_VAR_user_id

'7042'

In [9]:
import subprocess

command = f"cd {REPO_NAME}/infra && terraform init -backend-config=bucket=ds-2024l-{TF_VAR_user_id}-state"

result = subprocess.run(command, shell=True, check=True)
print("Komenda zakończona sukcesem", result)

Komenda zakończona sukcesem CompletedProcess(args='cd big-data-lab/infra && terraform init -backend-config=bucket=ds-2024l-7042-state', returncode=0)


### Provision the infrastructure

In [11]:
%%shell
cd $REPO_NAME/infra
terraform apply -auto-approve

Acquiring state lock. This may take a few moments...
module.vpc.module.vpc.module.vpc.google_compute_network.network: Refreshing state... [id=projects/ds-2024l-7042/global/networks/main-vpc]
module.vpc.module.vpc.module.subnets.google_compute_subnetwork.subnetwork["europe-west1/dataproc-01"]: Refreshing state... [id=projects/ds-2024l-7042/regions/europe-west1/subnetworks/dataproc-01]
module.vpc.google_compute_firewall.fw-allow-ingress-from-iap: Refreshing state... [id=projects/ds-2024l-7042/global/firewalls/fw-allow-ingress-iap]
module.vpc.google_compute_firewall.default-internal-allow-all: Refreshing state... [id=projects/ds-2024l-7042/global/firewalls/default-internal-allow-all]
module.vpc.module.cloud-router.google_compute_router.router: Refreshing state... [id=projects/ds-2024l-7042/regions/europe-west1/routers/nat-router]
module.vpc.module.vpc.module.routes.google_compute_route.route["egress-internet"]: Refreshing state... [id=projects/ds-2024l-7042/global/routes/egress-internet]


### Destroy the infrastructure
After you are done with the lab, you can destroy the infrastructure by running the cell below:

In [ ]:
%%shell
cd $REPO_NAME/infra
terraform destroy -auto-approve